<a href="https://colab.research.google.com/github/hwizard-wph/NAS/blob/main/Evolution_based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x
!curl -O https://storage.googleapis.com/nasbench/nasbench_only108.tfrecord
!git clone https://github.com/google-research/nasbench
!pip install ./nasbench
from nasbench import api
nasbench = api.NASBench('nasbench_only108.tfrecord')

TensorFlow 1.x selected.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  498M  100  498M    0     0   119M      0  0:00:04  0:00:04 --:--:--  119M
Cloning into 'nasbench'...
remote: Enumerating objects: 96, done.
remote: Total 96 (delta 0), reused 0 (delta 0), pack-reused 96
Unpacking objects: 100% (96/96), done.
Processing ./nasbench
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for nasbench: filename=nasbench-1.0-py3-none-any.whl size=46788 sha256=4b7a0336145b76a8104655f76a6e80e73eec3f04b58d9c303ee9f8a523a36326


In [2]:
import copy
import numpy as np
import matplotlib.pyplot as plt
import random

INPUT = 'input'
OUTPUT = 'output'
CONV3X3 = 'conv3x3-bn-relu'
CONV1X1 = 'conv1x1-bn-relu'
MAXPOOL3X3 = 'maxpool3x3'
NUM_VERTICES = 7
MAX_EDGES = 9
EDGE_SPOTS = NUM_VERTICES * (NUM_VERTICES - 1) / 2
OP_SPOTS = NUM_VERTICES - 2 
ALLOWED_OPS = [CONV3X3, CONV1X1, MAXPOOL3X3]
ALLOWED_EDGES = [0, 1] 

In [8]:
def random_spec():
  while True:
    matrix = np.random.choice(ALLOWED_EDGES, size=(NUM_VERTICES, NUM_VERTICES))
    matrix = np.triu(matrix, 1)
    ops = np.random.choice(ALLOWED_OPS, size=(NUM_VERTICES)).tolist()
    ops[0] = INPUT
    ops[-1] = OUTPUT
    spec = api.ModelSpec(matrix=matrix, ops=ops)
    if nasbench.is_valid(spec):
      return spec


In [ ]:
def mutate_spec(old_spec):
  

In [ ]:
def run_evolution_search(cycles = 20000, population_size = 50, tournament_size = 10):
  nasbench.reset_budget_counters()
  population = []
  history = []
  for _ in range(population_size):
    spec = random_spec()
    data = nasbench.query(spec)
    population.append((data['validation_accuracy'], spec))
    history.append((data['validation_accuracy'], spec))
  while len(history) < cycles:
    sample = []
    while len(sample) < tournament_size:
      candidate = np.random_choice(population)
      sample.append(candidate)
    parent = max(sample)
    child_spec = mutate_spec(parent[1])
    child_data = nasbench.query(child_spec)
    child = (child_data['validation_accuracy'], child_spec)
    population.append(child)
    history.append(child)
    population.pop(0)
  
  return history
